In [23]:
import tensorflow as tf

In [2]:
import pandas as pd 

In [4]:
#from tensorflow.keras import  *

In [7]:
from gensim.models.doc2vec import Doc2Vec

In [11]:
import jieba

In [8]:
vec_model = Doc2Vec.load('vec_model')

In [5]:
data = pd.read_csv('td.csv')

In [6]:
data

,comment,labels
0,千呼万唤始出来，,neutral
1,尼康的APSC小相机终于发布了，,neutral
2,COOLPIX A. 你怎么看呢？,neutral
3,我看，尼康是挤牙膏挤惯了啊，,neutral
4,1，外观既没有V1时尚，,negative
...,...,...
5773,你会发现X2真心不错。,neutral
5774,还没一只镜头沉，,neutral
5775,随时你都可以把它戴在身边，,neutral
5776,公文包就可以轻松放进去，,neutral


In [12]:
com = []
lab = []
for (i,j) in zip(data['comment'],data['labels']):
    com.append(vec_model.infer_vector(list(jieba.cut(i))))
    lab.append(j)
    

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.735 seconds.
Prefix dict has been built succesfully.


In [15]:
train_x = com[:5000]
train_y = lab[:5000]
test_x = com[5000:]
test_y = lab[5000:]


In [16]:
size =300

In [24]:

model = Sequential([
    tf.keras.Dense(input_shape = (size,)),
    tf.keras.Dense(256,activations = 'relu'),
    tf.keras.Dense(128),
    tf.keras.Dense(3,activations = 'softmax')
])

AttributeError: module 'tensorflow_core.keras' has no attribute 'Dense'

In [29]:
from sklearn.neural_network import MLPClassifier

In [30]:
model = sklearn.neural_network.MLPClassifier()

In [31]:
model.fit(train_x,train_y)


/home/gardra/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [32]:
pred_y = model.predict(test_x)

In [33]:
 from sklearn.metrics import confusion_matrix

In [34]:
confusion_matrix(test_y,pred_y)

array([[  6,  39,   6],
       [ 18, 535,  68],
       [  5,  94,   7]])

In [35]:
from sklearn.metrics import classification_report

In [36]:
print(classification_report(test_y,pred_y))

              precision    recall  f1-score   support

    negative       0.21      0.12      0.15        51
     neutral       0.80      0.86      0.83       621
    positive       0.09      0.07      0.07       106

    accuracy                           0.70       778
   macro avg       0.36      0.35      0.35       778
weighted avg       0.66      0.70      0.68       778



In [37]:
from collections import Counter

In [59]:
count = Counter()
for i in data['comment']:
    string = list(jieba.cut(i))
    for j in string:
        count.update({j})

In [60]:
len(count)

6339

In [61]:
len(list(count.elements()))

44174

In [46]:
hot = {}
j=0
for i in count.elements():
    hot[i]=j
    j+=1 

In [51]:
import numpy as np 

In [62]:
def bag(comments):
    re= []
    for i in comments:
        bag_word = np.zeros(44174)
        string = list(jieba.cut(i))
        for j in string:
            bag_word[hot[j]]=1
    
        re.append(bag_word)
    return re

In [72]:
train_x = bag(data['comment'][:100])
train_y = data['labels'][:100]

In [50]:
m

5000             大致都要1秒左右的时间。
5001          当然K01的优势在于对得更准。
5002         因此K01适合的拍摄范围是风光，
5003                静物和人物的摆拍。
5004    希望用K01去抓拍精彩瞬间的可能要失望了。
                ...          
5773              你会发现X2真心不错。
5774                 还没一只镜头沉，
5775            随时你都可以把它戴在身边，
5776             公文包就可以轻松放进去，
5777              想拍什么拿出来拍就是。
Name: comment, Length: 778, dtype: object

In [73]:
model = sklearn.neural_network.MLPClassifier()

In [74]:
model.fit(train_x,train_y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [75]:
test_x = bag(data['comment'][5000:])

In [76]:
pred_y = model.predict(test_x)

In [77]:
print(classification_report(test_y,pred_y))

              precision    recall  f1-score   support

    negative       0.09      0.14      0.11        51
     neutral       0.79      0.84      0.81       621
    positive       0.14      0.06      0.08       106

    accuracy                           0.69       778
   macro avg       0.34      0.34      0.33       778
weighted avg       0.66      0.69      0.67       778

